In [2]:
import pandas as pd
import numpy as np
import requests
import json

from config import gkey

ModuleNotFoundError: No module named 'config'

In [32]:
menu = pd.read_csv('Menu.csv', encoding="ISO-8859-1")
menuitem = pd.read_csv('MenuItem.csv', encoding="ISO-8859-1")
menupage = pd.read_csv('MenuPage.csv', encoding="ISO-8859-1")
dish = pd.read_csv('Dish.csv', encoding="ISO-8859-1")


In [33]:
merge_table1 = pd.merge(menu,menupage, left_on="id", right_on= 'menu_id',how="left")
merge_table1.count()

id_x                    61134
name                    10782
sponsor                 56498
event                   27884
venue                   28822
place                   28884
physical_description    51772
occasion                16056
notes                   40153
call_number             56492
keywords                    0
language                    0
date                    58421
location                61134
location_type               0
currency                20647
currency_symbol         20647
status                  61134
page_count              61134
dish_count              61134
id_y                    61134
menu_id                 61134
page_number             60189
image_id                61134
full_height             61134
full_width              61134
uuid                    61134
dtype: int64

In [34]:
merge_table1 = merge_table1[['event', 'place', 'occasion', 'dish_count', 'menu_id','date','location']]


In [35]:
final = pd.merge(merge_table1,menuitem, left_on="menu_id", right_on= 'menu_page_id',how="left")

In [36]:
final = pd.merge(final,dish, left_on="dish_id", right_on= 'id',how="left")

In [37]:
final.count()

event             335435
place             346903
occasion          202859
dish_count        710222
menu_id           710222
date              680145
location          710222
id_x              666762
menu_page_id      666762
price             273487
high_price         57259
dish_id           666605
created_at        666762
updated_at        666762
xpos              666762
ypos              666762
id_y              666605
name              666605
description            0
menus_appeared    666605
times_appeared    666605
first_appeared    666605
last_appeared     666605
lowest_price      631921
highest_price     631921
dtype: int64

In [38]:
final = final[['menu_id', 'event', 'place', 'occasion', 'dish_count', 'date','location', 'price', 'name', 'menus_appeared' ]]
final.head()

,menu_id,event,place,occasion,dish_count,date,location,price,name,menus_appeared
0,12463,BREAKFAST,"HOT SPRINGS, AR",EASTER;,67,1900-04-15,Hotel Eastman,NaN,NaN,NaN
1,12463,BREAKFAST,"HOT SPRINGS, AR",EASTER;,67,1900-04-15,Hotel Eastman,NaN,NaN,NaN
2,12464,[DINNER],"MILWAUKEE, [WI];",EASTER;,34,1900-04-15,Republican House,NaN,Erdbeeren,19.0
3,12464,[DINNER],"MILWAUKEE, [WI];",EASTER;,34,1900-04-15,Republican House,NaN,Aepfel,78.0
4,12464,[DINNER],"MILWAUKEE, [WI];",EASTER;,34,1900-04-15,Republican House,NaN,Apfelsinen,181.0


In [39]:
final.count()

menu_id           710222
event             335435
place             346903
occasion          202859
dish_count        710222
date              680145
location          710222
price             273487
name              666605
menus_appeared    666605
dtype: int64

In [23]:
final.to_csv("restaurant.csv", index=False, header=True)

3714

In [40]:
restaurant_places = final.groupby(['place'], as_index=False).mean()
restaurant_places.head()

,place,menu_id,dish_count,price,menus_appeared
0,"9 W 53rd St, New York, NY 10019",20966.000000,29.000000,NaN,933.800000
1,"""AUGUSTE VICTORIA""",26177.611429,18.285714,NaN,298.994253
2,"""DREAMLAND"", CONEY ISLAND, NY",22928.000000,11.000000,NaN,NaN
3,"""KONIGEN LOUISE"" AT SEA",26054.000000,76.000000,NaN,NaN
4,"""KONIGEN LUISE"" AT SEA",22144.176471,58.588235,NaN,217.562500


In [44]:
lat = []
lon = []
provinces = []

for place in restaurant_places['place']:
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?' 
    'address={0}&key={1}').format(place, gkey)
    response = requests.get(target_url)
    if response.ok:
        print(f'processing {place}')
        geo_data = response.json()
        if geo_data['results']:
            lat.append(geo_data["results"][0]["geometry"]["location"]["lat"])
            lon.append(geo_data["results"][0]["geometry"]["location"]["lng"])
            provinces.append(place)
        else:
            print('Skipping %s because no data'%place)
    else:
        print('Got error on %s'%target_url)
        print(response)

processing  9 W 53rd St, New York, NY 10019
processing "AUGUSTE VICTORIA"
Skipping "AUGUSTE VICTORIA" because no data
processing "DREAMLAND", CONEY ISLAND, NY
processing "KONIGEN LOUISE" AT SEA
Skipping "KONIGEN LOUISE" AT SEA because no data
processing "KONIGEN LUISE" AT SEA
Skipping "KONIGEN LUISE" AT SEA because no data
processing "KONIGIN LUISE"
Skipping "KONIGIN LUISE" because no data
processing "KONIGIN LUISE" AT SEA
Skipping "KONIGIN LUISE" AT SEA because no data
processing "KONIGIN LUSE" AT SEA
Skipping "KONIGIN LUSE" AT SEA because no data
processing "LITTLE HUNGARY" 257 EAST HOUSTON STREET, NEW YORK, NY
processing "LITTLE HUNGARY" [NYC]
processing "NEW YORK"
processing "ST. LOUIS"
processing "ST.LOUIS"
processing "THE ARGYLE"
processing "THE ARGYLE" 308 FULTON ST.,BROOKLYN,NY
processing "THE CLUB," NEW YORK, NY
processing "THE D&H" DINING CAR SERVICE BETWEEN ROUSES POINT, HOTEL CHAMPLAIN AND WHITEHALL
processing "WARREN"
processing 'SPRINGSIDE",ORANGE, NJ;
processing (3 WILLI

processing 58TH ST. AND LEXINGTON AVE., NY
processing 59 ST. & 5TH AVE. NY
processing 59-61 WEST 24TH STREET, NEW YORK [NY];
processing 59TH ST.& 5TH AVE.NY;
processing 59TH STREET AND BROADWAY, NEW YORK [NY];
processing 5TH AVE. & 15 ST. NY
processing 5TH AVE. & 15TH ST. NY
processing 6 EAST 23 ST. NY
processing 6 PARK PLACE
processing 6 PARK PLACE, NEW YORK
processing 6 PARK PLACE, NYC
processing 60 West 55th Street
processing 600 SIXTH AVE. NY
processing 602 SIXTH AVENUE, NEW YORK
processing 61 CORTLANDT ST.,NY
processing 61-63 EAST 59 STREET,NEW YORK,NY
processing 642 FIFTH AVE,(NY)
processing 66 ST. & B'WAY NY
processing 66 ST. & B'WAY. NY
processing 66 ST. & BWAY. NY
processing 663 BROADWAY,NY
processing 663-665 BROADWAY
processing 667 BOND ST. NY
processing 66TH ST & B'WAY, NY
processing 66TH ST AND BROADWAY,NEW YORK,NY
processing 66TH ST. & BWAY NY
processing 66TH ST. AND BROADWAY;NEW YORK,NY
processing 66TH STREET AND BR6ADWAY
processing 66TH STREET AND BROADWAY [NY]
processin

processing ASYLUM, MASONIC TEMPLE, BALTIMORE, MD
processing AT SEA
processing AT THE SPONSOR'S HOME IN HARRISBURG, PA
processing ATANTIC HOTEL, OCEAN CITY, MD
processing ATLANTA
processing ATLANTA, GA.
processing ATLANTIC CITY, N.J.
processing ATLANTIC CITY, NJ
processing ATLANTIC CITY, NJ;
processing ATLANTIC CITY,NJ
processing ATLANTIC CITY,[NJ]
processing ATLANTIC HOTEL, OCEAN CITY, MD
processing AUBERGE LAFAYETTE, PHILADELPHIA, PA
processing AUCKLAND,NEW ZEALEND;
processing AUDITORIUM
processing AUDITORIUM ANNEX
processing AUDITORIUM ANNEX HOTEL, [CHICAGO, IL]
processing AUDITORIUM ANNEX [CHICAGO, IL]
processing AUDITORIUM HOEL, CHICAO, IL
processing AUDITORIUM HOEL;
processing AUDITORIUM HOTEL
processing AUDITORIUM HOTEL , CHICAGO, IL
processing AUDITORIUM HOTEL ANNEX, (?), IL
processing AUDITORIUM HOTEL [CHICAGO IL]
processing AUDITORIUM HOTEL [CHICAGO, IL]
processing AUDITORIUM HOTEL, (?CHICAGO, IL?)
processing AUDITORIUM HOTEL, (CHICAGO, IL?)
processing AUDITORIUM HOTEL, CHICAG

processing BUDAPEST, (HUNGARY)
processing BUDAPEST, HUNGARY
processing BUFFALO NY(?)
processing BUFFALO, NY
processing BUFFALO,NY
processing BUFFET-RESTAURANT DE LA GARE, [BRIGUE?], SWITZERLAND
processing BURLANDO'S PROVIDENCE, RI;
processing BURLINGTON HOUSE, LONDON, ENGLAND
processing BURMA
processing BURNET HOUSE, [OHIO]
processing BURNS HOTEL,NYC
processing BUSHWICK CLUB, BUSHWICK AVE. AND HART ST., BROOKLYN, NY
processing BUSINESS MEN'S CLUB, CINCINNATI, OH
processing BUTLER, PA
processing BUTLER, PA.
processing BUTTE,MONTANA
processing BWAY & 63RD ST. NY
processing Baguita Trattoria Toscana
Skipping Baguita Trattoria Toscana because no data
processing Bali
processing Balthazar
processing Bar Angleterre
processing Bar de l'Hotel Royal
processing Bareschuur
processing Barriere de Clichy
processing Bayerischer Donisl
processing Bellini
processing Benrather Hof
processing Bentley's
processing Berkeley
processing Beverly Hills Hotel
processing Bier-halle Kropf
processing Biltmore Bowl

processing CRITERION RESTAURANT, VICTORIA HALL, LONDON, ENGLAND
processing CRITERION, PICCADILLY [ENG]
processing CROCKER HOUSE, NEW LONDON,CONN.
processing CRYSTAL PALACE, LONDON, ENGLAND
processing CURAZAO
processing Caesar's
processing Cafe "Alte Bastion"
processing Cafe Carlyle
processing Cafe Chauveron
Skipping Cafe Chauveron because no data
processing Cafe Imperial
processing Cafe Martin
processing Cafe Promenade
processing Cafe Royal
processing Cafe at the Seelbach
processing Cafe de Paris
processing Cafe del Sol
processing Cafe zur Frauenkirche Restaurant, An der Frauenkirche 7
processing Cafe-Restaurant "De Prinsenkelder"
processing California Building, Panama Pacific International Exposition
processing Candlelight Restaurant; 114 Henry Street
processing Canlis'
processing Capital Hotel
processing Carte
processing Cathay House
processing Cavanagh's
processing Cellini
processing Chanterelle
processing Charley Brown's
processing Charlie Brown's
processing Chateau de Brindos
proc

...

processing SS COPTIC
Skipping SS COPTIC because no data
processing SS DEUTSCHLAND
Skipping SS DEUTSCHLAND because no data
processing SS DORIC
Skipping SS DORIC because no data
processing SS DORIC;
Skipping SS DORIC; because no data
processing SS ETRURIA
Skipping SS ETRURIA because no data
processing SS FRIEDRICH DER GROSSE
Skipping SS FRIEDRICH DER GROSSE because no data
processing SS FRIESLAND
Skipping SS FRIESLAND because no data
processing SS FURST BISMARCK
Skipping SS FURST BISMARCK because no data
processing SS FURST BISMARK
Skipping SS FURST BISMARK because no data
processing SS GAELIC
Skipping SS GAELIC because no data
processing SS GRAF WALDERSEE
Skipping SS GRAF WALDERSEE because no data
processing SS HOHENZOLLERN
Skipping SS HOHENZOLLERN because no data
processing SS HONG KONG MARU
Skipping SS HONG KONG MARU because no data
processing SS KAISER WILHELM DER GROSSE
Skipping SS KAISER WILHELM DER GROSSE because no data
processing SS KAMAKUKRA MARU
Skipping SS KAMAKUKRA MARU beca

processing Schlosshotel Kronberg
processing Schultheiss
processing Schutzenhof
Skipping Schutzenhof because no data
processing Schwarzwalders Natur Weinhaus
processing Scott's Restaurant
processing Seasons
processing Sebastians
processing Seehotel Uberfahrt
processing Senor Pico; Ghiradelli Square
processing Serendipity 3; 225 East 60th Street
processing Shaul's Inn
processing Sheraton
processing Shopsy's
processing Shun Lee West
processing Silvermine Tavern
processing Simpson's-in-the-Strand
processing Sirloin Room
processing Sitar
processing Sonnenberg
processing Southern Pacific
processing Southern Pacific Company, SUNSET OGDEN & SHASTA ROUTES
processing Spatenhaus an der Oper
processing Sprungli
processing St. Regis Hotel
processing Stalter Hilton; Hotel Statler
processing Stammhaus
Skipping Stammhaus because no data
processing Star of the Sea Room
processing Statler Hilton
processing Statler Hilton; The Pen and Sword
processing Steak House zum Franziskaner
processing Steigenberger

processing TREMONT TEMPLE,BOSTON MASS
processing TROCADERO
processing TROCADERO, [NEW YORK,N.Y.?]
processing TROCADERO;PROVIDENCE,RI.
processing TROMMER'S HALL, BROOKLYN, NY
processing TRONDHJEM (NOWAY?)
processing TURISTHOTELLER, DINING ROOM OF FAIRY TALES, NORWAY
Skipping TURISTHOTELLER, DINING ROOM OF FAIRY TALES, NORWAY because no data
processing Tadich Grill; 545 Clay Street
processing Tait's "Pavo Real"
Skipping Tait's "Pavo Real" because no data
processing Tao Tao Restaurant
processing Tao Yuan Restaurant
processing Tarasco Grill
processing Taverne Provence
processing Templebar; No. 1 Tillman Place
processing Terrace Restaurant
processing Terrass
processing The Beaver, Southern Pacific
Skipping The Beaver, Southern Pacific because no data
processing The Boar's Head Inn
processing The Box Tree; 250 East 49th Street
processing The Buena Vista
processing The Carlyle
processing The Carlyle; Madison Avenue at 76th Street
processing The Chinese Rathskeller
processing The Chronicle
pro

processing WESTMINSTER HOTEL
processing WESTMINSTER HOTEL, [NEW YORK, NY?]
processing WETHERSFIELD, CT
processing WHEATON, MD
processing WHITE HOUSE HOTEL,[STATEN ISLAND,NY?]
processing WHITE MOUNTAINS, NH
processing WHITE MOUNTAINS, NH;
processing WHITE MOUNTAINS,N.H.
processing WHITE'S TONTINE HOTEL, NEW HAVEN, CT
processing WHITECHAPEL ROAD, LONDON, ENGLAND
processing WHITEHALL PLACE, LONDON, ENGLAND
processing WHITEHALL ROOMS, THE HOTEL METROPOLE, LONDON [ENGLAND]
processing WHITEHALL ROOMS, THE HOTEL METROPOLE, LONDON, [ENGLAND]
processing WHITEHALL ROOMS,THE HOTEL METROPOLE, LONDON,ENGLAND
processing WILKES-BARRE, PA;
processing WILLARD HOTEL, [WASHINGTON, DC]
processing WILLARD HOTEL,WASH.D.C.
processing WILLARD'S HOTEL WASH.DC;
processing WILLIS'S ROOMS, LONDON, ENGLAND
processing WILLOW GROVE PARK, PA
processing WILMINGTON NC
processing WILTSHIRE HOTEL,ATLANTIC CITY,NJ
processing WINDSOR CASTLE
processing WINDSOR CASTLE, ENGLAND
processing WINDSOR CASTLE, [ENGLAND]
processing 

processing restaurant, l'Excuse, 14 Rue Charles V 75004, France
processing resturant, Le Vert Galant, France
processing unknown


In [45]:
geo_dict = {
   "place": provinces,
    "lat": lat,
    "lon": lon
}
geo_df = pd.DataFrame(geo_dict)
full_set = final.merge(geo_df, how="left", on = "place")
full_set = full_set.dropna(subset=['lat'])
full_set.head()

,menu_id,event,place,occasion,dish_count,date,location,price,name,menus_appeared,lat,lon
0,12463,BREAKFAST,"HOT SPRINGS, AR",EASTER;,67,1900-04-15,Hotel Eastman,NaN,NaN,NaN,34.503700,-93.055179
1,12463,BREAKFAST,"HOT SPRINGS, AR",EASTER;,67,1900-04-15,Hotel Eastman,NaN,NaN,NaN,34.503700,-93.055179
2,12464,[DINNER],"MILWAUKEE, [WI];",EASTER;,34,1900-04-15,Republican House,NaN,Erdbeeren,19.0,43.038902,-87.906474
3,12464,[DINNER],"MILWAUKEE, [WI];",EASTER;,34,1900-04-15,Republican House,NaN,Aepfel,78.0,43.038902,-87.906474
4,12464,[DINNER],"MILWAUKEE, [WI];",EASTER;,34,1900-04-15,Republican House,NaN,Apfelsinen,181.0,43.038902,-87.906474


In [47]:
full_set = full_set.dropna(subset=['name'])

In [48]:
full_set.count()

menu_id           273858
event             250225
place             273858
occasion          166561
dish_count        273858
date              249166
location          273858
price              76224
name              273858
menus_appeared    273858
lat               273858
lon               273858
dtype: int64

In [64]:
#full_set.groupby('name')['lat', 'lon'].nunique()
full_set['rnd_lat'] = full_set['lat'].round(0)
full_set['rnd_lon'] = full_set['lon'].round(0)
full_set.dtypes
#df = full_set.groupby(['name','month'])['text'].apply(','.join).reset_index()

menu_id             int64
event              object
place              object
occasion           object
dish_count          int64
date               object
location           object
price             float64
name               object
menus_appeared    float64
lat               float64
lon               float64
rnd_lat           float64
rnd_lon           float64
dtype: object

In [69]:
df = full_set.groupby(['rnd_lat','rnd_lon'])['name'].apply(' '.join).reset_index()

In [70]:
df

,rnd_lat,rnd_lon,name
0,-38.0,145.0,"Consomme with tapioca Striped Bass, Anchovy sa..."
1,-37.0,175.0,HuÃ®tres Royales Hors d'Åuvre variÃ©s Consomm...
2,-35.0,139.0,Strawberries Jam Muskmelon Radishes Watercress...
3,-25.0,134.0,Olives Caviar Sliced Tomatoes Lobster salad Bo...
4,-23.0,-43.0,Claret Cup Rhine or Mosel Cup (Bowle) CHampagn...
5,-8.0,115.0,"Blue Points, Mignonette Celery Russian Caviar ..."
6,7.0,80.0,Zuckermelone Pompelmuse Bananen Hafergrutze Mi...
7,14.0,101.0,Consomme mit Griesnocken Wildpretsuppe Lachs-S...
8,14.0,121.0,Tomatoes Apples Grape fruit Tomaten Apfel Pamp...
9,15.0,121.0,Sardellen-Eier Spargel-Suppe Fleischbruhe Mitt...


In [71]:
from collections import Counter

In [72]:
# split_it = .split() 
  
# # Pass the split_it list to instance of Counter class. 
# Counter = Counter(split_it) 
  
# # most_common() produces k frequently encountered 
# # input values and their respective counts. 
# most_occur = Counter.most_common(4) 
  
# print(most_occur) 

pd.Series(' '.join(df.name).lower().split()).value_counts()[:100]

with         9853
sauce        9564
potatoes     9543
of           8943
a            8683
beef         8033
la           7914
and          7441
cream        6901
&            6630
roast        6311
fried        6265
boiled       6264
cheese       6154
chicken      5154
coffee       5120
salad        4951
broiled      4875
de           4730
cold         4625
ham          4188
pie          3843
tea          3780
au           3499
rice         3463
eggs         3436
baked        3237
mutton       3136
corned       3039
cakes        2928
             ... 
corn         1423
sweet        1416
en           1399
fruits       1396
chocolate    1391
oysters      1381
mushrooms    1364
apples       1362
american     1351
sliced       1346
spring       1315
new          1312
lobster      1299
butter       1275
creme        1253
crackers     1234
smoked       1214
plain        1212
potatoes,    1202
per          1201
hot          1188
pickled      1182
stuffed      1176
sausage      1175
hash      

In [68]:
counts = df
counts['most_common']= df['name'].most_common(3)

AttributeError: 'Series' object has no attribute 'most_common'

In [75]:
res = df['name'].str.split().apply(Counter)
res

0      {'Consomme': 4, 'with': 8, 'tapioca': 4, 'Stri...
1      {'HuÃ®tres': 2, 'Royales': 2, 'Hors': 2, 'd'Å...
2      {'Strawberries': 2, 'Jam': 2, 'Muskmelon': 2, ...
3      {'Olives': 4, 'Caviar': 4, 'Sliced': 4, 'Tomat...
4      {'Claret': 8, 'Cup': 6, 'Rhine': 2, 'or': 14, ...
5      {'Blue': 4, 'Points,': 4, 'Mignonette': 4, 'Ce...
6      {'Zuckermelone': 2, 'Pompelmuse': 2, 'Bananen'...
7      {'Consomme': 8, 'mit': 8, 'Griesnocken': 4, 'W...
8      {'Tomatoes': 4, 'Apples': 4, 'Grape': 4, 'frui...
9      {'Sardellen-Eier': 4, 'Spargel-Suppe': 4, 'Fle...
10     {'Chicken-broth': 4, 'with': 8, 'Veal-dumpling...
11     {'Clovis': 4, 'Consomme': 4, 'de': 16, 'Tortue...
12     {'consomme,': 2, 'Spaghetti': 2, 'Fried': 2, '...
13     {'Puchero': 4, 'Heilbutt-Steak': 2, 'mit': 2, ...
14     {'Potage': 2, 'creme': 2, 'chatelaine': 2, 'Sa...
15     {'Mutton': 4, 'croquettes': 4, 'String': 4, 'B...
16     {'Fruit': 2, 'hominy': 2, 'with': 2, 'milk': 2...
17     {'Fruit': 8, 'Oat': 4, '

In [87]:
import operator
from heapq import nlargest

In [88]:
#results = [max(x, key=x.get) for x in res]
results = [nlargest(5, x, key=x.get) for x in res]
results

[['Champagne,', 'Liqueurs,', 'Mineral', 'Waters,', '&'],
 ['de', 'Ã', 'la', 'HuÃ®tres', 'Royales'],
 ['Broiled', 'mutton', 'with', 'Strawberries', 'Jam'],
 ['of', 'Sauce', 'Green', 'Olives', 'Caviar'],
 ['with', 'of', 'Steak', 'Fried', 'Eggs'],
 ['of', 'en', 'au', 'Sur', 'Cream'],
 ['Cheese', 'a', 'la', 'Fried', 'Coffee'],
 ['and', 'Consomme', 'mit', 'a', 'la'],
 ['Beefsteak,', 'au', 'mit', 'potatoes', 'Boiled'],
 ['Roast', 'Succotash', 'with', '&', 'in'],
 ['with', 'Green', 'style', 'Chicken-broth', 'Veal-dumplings'],
 ['de', 'Salade', 'Clovis', 'Consomme', 'Tortue'],
 ['beef', 'salad', 'cream', 'consomme,', 'Spaghetti'],
 ['Puchero', 'Heilbutt-Steak', 'mit', 'feinen', 'Krautern'],
 ['a', 'la', 'en', 'du', 'sauce'],
 ['Club', 'Old', 'St.', 'or', 'Rye'],
 ['Broiled', '(to', 'order)', 'Fried', 'Cakes'],
 ['and', 'with', 'of', 'Potatoes', 'Beef'],
 ['Cold', 'Eggs', 'Chicken', 'Fried', 'Apple'],
 ['Roast', 'of', 'Chicken', 'beans', 'Potatoes'],
 ['a', 'la', 'of', 'and', 'de'],
 ['with', '

In [89]:
len(results)

262

In [90]:
df['top_words']=results

In [91]:
df.head()

,rnd_lat,rnd_lon,name,top_words
0,-38.0,145.0,"Consomme with tapioca Striped Bass, Anchovy sa...","[Champagne,, Liqueurs,, Mineral, Waters,, &]"
1,-37.0,175.0,HuÃ®tres Royales Hors d'Åuvre variÃ©s Consomm...,"[de, Ã, la, HuÃ®tres, Royales]"
2,-35.0,139.0,Strawberries Jam Muskmelon Radishes Watercress...,"[Broiled, mutton, with, Strawberries, Jam]"
3,-25.0,134.0,Olives Caviar Sliced Tomatoes Lobster salad Bo...,"[of, Sauce, Green, Olives, Caviar]"
4,-23.0,-43.0,Claret Cup Rhine or Mosel Cup (Bowle) CHampagn...,"[with, of, Steak, Fried, Eggs]"


In [22]:
df.to_csv("restaurant_location_words.csv", index=False, header=True)

NameError: name 'df' is not defined

In [25]:
mapready = pd.read_csv('restaurant_location_words.csv', encoding="ISO-8859-1")

In [26]:
mapready['location']= mapready[['rnd_lat','rnd_lon']].values.tolist()
mapready = mapready[['location','top_words']]
mapready

,location,top_words
0,"[-38.0, 145.0]","['Champagne,', 'Liqueurs,', 'Mineral', 'Waters..."
1,"[-37.0, 175.0]","['de', 'Ã', 'la', 'HuÃÂ®tres', 'Royales']"
2,"[-35.0, 139.0]","['Broiled', 'mutton', 'with', 'Strawberries', ..."
3,"[-25.0, 134.0]","['of', 'Sauce', 'Green', 'Olives', 'Caviar']"
4,"[-23.0, -43.0]","['with', 'of', 'Steak', 'Fried', 'Eggs']"
5,"[-8.0, 115.0]","['of', 'en', 'au', 'Sur', 'Cream']"
6,"[7.0, 80.0]","['Cheese', 'a', 'la', 'Fried', 'Coffee']"
7,"[14.0, 101.0]","['and', 'Consomme', 'mit', 'a', 'la']"
8,"[14.0, 121.0]","['Beefsteak,', 'au', 'mit', 'potatoes', 'Boiled']"
9,"[15.0, 121.0]","['Roast', 'Succotash', 'with', '&', 'in']"


In [34]:
mapready.to_dict('records')

[{'location': [-38.0, 145.0],
  'top_words': "['Champagne,', 'Liqueurs,', 'Mineral', 'Waters,', '&']"},
 {'location': [-37.0, 175.0],
  'top_words': "['de', 'Ã\x83', 'la', 'HuÃ\x83Â®tres', 'Royales']"},
 {'location': [-35.0, 139.0],
  'top_words': "['Broiled', 'mutton', 'with', 'Strawberries', 'Jam']"},
 {'location': [-25.0, 134.0],
  'top_words': "['of', 'Sauce', 'Green', 'Olives', 'Caviar']"},
 {'location': [-23.0, -43.0],
  'top_words': "['with', 'of', 'Steak', 'Fried', 'Eggs']"},
 {'location': [-8.0, 115.0],
  'top_words': "['of', 'en', 'au', 'Sur', 'Cream']"},
 {'location': [7.0, 80.0],
  'top_words': "['Cheese', 'a', 'la', 'Fried', 'Coffee']"},
 {'location': [14.0, 101.0],
  'top_words': "['and', 'Consomme', 'mit', 'a', 'la']"},
 {'location': [14.0, 121.0],
  'top_words': "['Beefsteak,', 'au', 'mit', 'potatoes', 'Boiled']"},
 {'location': [15.0, 121.0],
  'top_words': "['Roast', 'Succotash', 'with', '&', 'in']"},
 {'location': [18.0, -70.0],
  'top_words': "['with', 'Green', 'sty